# Spark ML: Overview

Spark ML (Machine Learning) adalah pustaka untuk pembelajaran mesin di Apache Spark. Pustaka ini dirancang untuk memudahkan pengembangan, penerapan, dan pengelolaan algoritma pembelajaran mesin pada data besar. 

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [ ]:
sc =SparkContext()
spark = SparkSession.builder.appName("Python Spark ML basic example").getOrCreate()

## Diagnostic Analytics

Jenis diagnostic analytics berguna ketika lembaga, organisasi, atau perusahaan ingin mendapatkan wawasan mengenai masalah tertentu. Proses analisis dilakukan dengan melakukan pemulihan, pengembangan, dan penelusuran data. Data yang dimasukkan dalam analisis tentu saja lebih banyak dan bervariasi. 

* Tujuan: Menganalisis data untuk memahami penyebab di balik kejadian atau pola yang ditemukan.
* Metode: Menggunakan teknik analisis lebih mendalam seperti analisis sebab-akibat dan regresi.

Contoh: Menyelidiki penurunan performa penjualan dengan mencari tahu penyebabnya, seperti perubahan dalam strategi pemasaran atau faktor eksternal.

Metode analisis yg digunakan:
* Correlation: Mengukur kekuatan dan arah hubungan antara dua variabel dengan korelasi Pearson atau Spearman.
* ChiSquareTest: Menguji independensi antara fitur dan label pada data kategorikal menggunakan uji chi-square.
* Summarizer: Menghitung statistik deskriptif seperti rata-rata dan jumlah untuk fitur, baik dengan atau tanpa pembobotan.

### Correlation

Mengukur kekuatan dan arah hubungan antara dua variabel.

* Korelasi Pearson: Mengukur hubungan linier antara dua variabel. Nilai berkisar dari -1 hingga 1, di mana 1 menunjukkan hubungan linier positif sempurna, -1 menunjukkan hubungan linier negatif sempurna, dan 0 menunjukkan tidak ada hubungan linier.
* Korelasi Spearman: Mengukur hubungan monotonic antara dua variabel. Ini tidak memerlukan hubungan linier dan cocok untuk data ordinal atau data yang tidak memenuhi asumsi linearitas.

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

In [ ]:
data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
data

* Vectors.sparse: Membuat vektor fitur yang jarang (sparse vector) dengan dimensi 4. Misalnya, Vectors.sparse(4, [(0, 1.0), (3, -2.0)]) berarti vektor dengan nilai 1.0 pada indeks 0 dan -2.0 pada indeks 3, sedangkan nilai lainnya adalah 0.
* Vectors.dense: Membuat vektor fitur yang padat (dense vector). Misalnya, Vectors.dense([4.0, 5.0, 0.0, 3.0]) berarti vektor dengan nilai 4.0, 5.0, 0.0, dan 3.0 pada indeks yang sesuai.

Pemilihan antara vektor jarang dan padat bergantung pada karakteristik data dan efisiensi yang diperlukan untuk pemrosesan dan penyimpanan.

* Vektor jarang adalah representasi dari vektor di mana sebagian besar elemen memiliki nilai nol. Hanya elemen-elemen yang tidak nol yang disimpan secara eksplisit.
  * Vektor jarang hanya menyimpan nilai-nilai yang tidak nol dan posisi mereka. Ini menghemat memori, terutama untuk vektor dengan banyak elemen nol.
  * Keuntungan: menggunakan lebih sedikit memori untuk vektor dengan banyak elemen nol, operasi matematis seperti perkalian matriks dapat lebih efisien pada vektor jarang.
  * Digunakan bila data dengan banyak nilai nol, seperti representasi teks atau fitur dengan banyak kategori yang tidak aktif.
* Vektor padat adalah representasi dari vektor di mana semua elemen, termasuk yang bernilai nol, disimpan secara eksplisit.
  * Semua elemen dari vektor disimpan, tanpa memandang apakah mereka nol atau tidak. Ini memerlukan lebih banyak memori dibandingkan dengan vektor jarang jika banyak elemen adalah nol.
  * Keuntungan: format yang lebih sederhana dan sering digunakan dalam algoritma yang memerlukan akses langsung ke setiap elemen, beberapa operasi matematis dan algoritma mungkin lebih mudah diterapkan pada vektor padat.
  * Digunakan bila data yang tidak memiliki banyak elemen nol, atau ketika memori dan efisiensi penyimpanan tidak menjadi masalah besar.

In [ ]:
df = spark.createDataFrame(data, ["features"])
df.show()

* spark.createDataFrame(data, ["features"]): Membuat DataFrame dari data yang berisi kolom "features". DataFrame ini akan memiliki satu kolom yang berisi vektor fitur.

In [ ]:
r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

* Correlation.corr(df, "features"): Menghitung matriks korelasi Pearson untuk kolom "features" dari DataFrame df. Fungsi ini mengembalikan matriks korelasi yang menunjukkan sejauh mana fitur dalam vektor fitur saling berkorelasi.
* .head(): Mengambil baris pertama dari hasil matriks korelasi.
* print("Pearson correlation matrix:\n" + str(r1[0])): Menampilkan matriks korelasi Pearson dalam format string. r1[0] berisi matriks korelasi dalam bentuk Row PySpark.

In [ ]:
r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))

* Correlation.corr(df, "features", "spearman"): Menghitung matriks korelasi Spearman untuk kolom "features" dari DataFrame df. Korelasi Spearman mengukur kekuatan dan arah hubungan monotonic antara dua variabel.
  * df: DataFrame yang berisi kolom "features" dengan vektor fitur.
  * "features": Nama kolom dalam DataFrame yang berisi vektor fitur yang akan dihitung korelasinya.
  * "spearman": Jenis korelasi yang digunakan adalah korelasi Spearman. Korelasi Spearman berbeda dari korelasi Pearson karena mengukur hubungan monotonic dan tidak memerlukan asumsi linearitas.
* print("Spearman correlation matrix:\n" + str(r2[0])): Menampilkan matriks korelasi Spearman yang telah dihitung. r2[0] berisi matriks korelasi dalam bentuk DenseMatrix yang diubah menjadi string untuk ditampilkan.

### Chi Square

 Mengukur independensi antara fitur dan label dalam data kategorikal. Uji chi-square menguji apakah distribusi fitur berbeda secara signifikan berdasarkan label.

 * Uji Chi-Square: Berguna untuk data kategorikal untuk menentukan apakah ada asosiasi atau ketergantungan antara variabel kategorikal.
* Statistik: Menghasilkan nilai p, derajat kebebasan, dan statistik chi-square yang digunakan untuk menentukan signifikansi hubungan antara fitur dan label.

Contoh Penggunaan Uji Chi-Square

* Menilai Asosiasi: Uji chi-square digunakan untuk menilai apakah ada asosiasi signifikan antara fitur kategorikal dan variabel target. Ini berguna dalam analisis data untuk memahami hubungan antara fitur dan target.
* Penerapan dalam Model: Uji ini bisa digunakan dalam pemilihan fitur untuk model pembelajaran mesin untuk memilih fitur yang signifikan secara statistik.

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import ChiSquareTest

data = [(0.0, Vectors.dense(0.5, 10.0)),
        (0.0, Vectors.dense(1.5, 20.0)),
        (1.0, Vectors.dense(1.5, 30.0)),
        (0.0, Vectors.dense(3.5, 30.0)),
        (0.0, Vectors.dense(3.5, 40.0)),
        (1.0, Vectors.dense(3.5, 40.0))]
df = spark.createDataFrame(data, ["label", "features"])

In [ ]:
df.show()

* Vectors.dense(): Membuat vektor fitur padat (dense vector) untuk setiap baris data. Misalnya, Vectors.dense(0.5, 10.0) adalah vektor dengan nilai 0.5 dan 10.0.
* spark.createDataFrame(data, ["label", "features"]): Membuat DataFrame df dengan kolom "label" dan "features". Kolom "label" adalah variabel target, dan kolom "features" adalah fitur yang digunakan untuk uji chi-square.

In [ ]:
r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

* ChiSquareTest.test(df, "features", "label"): Melakukan uji chi-square untuk mengukur ketergantungan antara fitur ("features") dan label ("label"). Uji ini mengevaluasi apakah distribusi fitur berbeda secara signifikan berdasarkan label.
* r.pValues: Nilai p untuk uji chi-square, menunjukkan signifikansi statistik dari hubungan antara fitur dan label. Nilai p yang kecil menunjukkan bahwa ada hubungan signifikan antara fitur dan label.
* r.degreesOfFreedom: Derajat kebebasan dari uji chi-square, menunjukkan jumlah parameter yang dapat bervariasi bebas dalam uji.
* r.statistics: Statistik chi-square untuk setiap fitur, menunjukkan ukuran ketergantungan antara fitur dan label.

### Summarizer

Menghitung statistik deskriptif seperti rata-rata, jumlah, dan lainnya untuk fitur dalam data, baik dengan atau tanpa pembobotan.

* Summarizer: Berguna untuk menghitung statistik seperti rata-rata (mean), jumlah (count), dan variansi (variance) untuk fitur vektor. Ini dapat dilakukan dengan mempertimbangkan pembobotan jika diperlukan.
* Statistik: Memungkinkan perhitungan statistik deskriptif untuk setiap fitur vektor secara efisien.

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

df = sc.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
                     Row(weight=0.0, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()
df.show()

* sc.parallelize(): Mengonversi data menjadi RDD dan kemudian membuat DataFrame dari RDD tersebut.
* Row(weight=..., features=...): Membuat baris data dengan kolom weight dan features. features adalah vektor fitur padat (dense vector).

In [ ]:
from pyspark.ml.stat import Summarizer

# create summarizer for multiple metrics "mean" and "count"
summarizer = Summarizer.metrics("mean", "count")

# compute statistics for multiple metrics with weight
df.select(summarizer.summary(df.features, df.weight)).show(truncate=False)

* Summarizer.metrics("mean", "count"): Membuat objek Summarizer yang akan menghitung rata-rata ("mean") dan jumlah ("count") untuk setiap vektor fitur.
* df.select(summarizer.summary(df.features, df.weight)): Menghitung statistik untuk fitur dengan memperhitungkan bobot. summarizer.summary menghasilkan rata-rata dan jumlah berdasarkan bobot yang diberikan.

In [ ]:
# compute statistics for multiple metrics without weight
df.select(summarizer.summary(df.features)).show(truncate=False)

* df.select(summarizer.summary(df.features)): Menghitung statistik untuk fitur tanpa mempertimbangkan bobot. Ini menghasilkan rata-rata dan jumlah berdasarkan nilai fitur langsung.

In [ ]:
# compute statistics for single metric "mean" with weight
df.select(Summarizer.mean(df.features, df.weight)).show(truncate=False)

* Summarizer.mean(df.features, df.weight): Menghitung rata-rata (mean) dari fitur dengan mempertimbangkan bobot. Hasilnya adalah rata-rata tertimbang dari fitur.

In [ ]:
# compute statistics for single metric "mean" without weight
df.select(Summarizer.mean(df.features)).show(truncate=False)

* Summarizer.mean(df.features): Menghitung rata-rata (mean) dari fitur tanpa mempertimbangkan bobot. Ini adalah rata-rata sederhana dari fitur.

### Analisis Teks

Pipeline ini terdiri dari beberapa tahap, termasuk tokenisasi, vektorisasi hitung (Count Vectorization), dan perhitungan IDF (Inverse Document Frequency).

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0, "Python python Spark Spark"),
    (1, "Python SQL")],
 ["document", "sentence"])

* spark.createDataFrame([...], ["document", "sentence"]): Membuat DataFrame sentenceData dengan dua kolom: "document" dan "sentence". Kolom "sentence" berisi teks yang akan diproses.

In [ ]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
vectorizer  = CountVectorizer(inputCol="words", outputCol="rawFeatures")

* Tokenisasi: Berguna untuk memecah teks menjadi kata-kata individu untuk analisis lebih lanjut.
  * Tokenizer: Tokenizer membagi kalimat dalam kolom "sentence" menjadi daftar kata-kata individu. Outputnya adalah kolom "words".
* CountVectorizer: Mengonversi teks menjadi representasi numerik yang dapat digunakan untuk pembelajaran mesin. 
  * CountVectorizer: Mengubah daftar kata-kata ("words") menjadi vektor fitur hitung (count vector). Outputnya adalah kolom "rawFeatures", di mana setiap kata dalam dokumen dipetakan ke vektor hitung.

In [ ]:
idf = IDF(inputCol="rawFeatures", outputCol="features")

* IDF: Menyaring fitur penting dari fitur yang jarang muncul, meningkatkan kualitas model pembelajaran mesin dengan memberikan bobot yang lebih besar pada kata-kata yang lebih informatif.
  * Fungsi IDF: Menghitung nilai IDF dari vektor hitung. IDF digunakan untuk mengukur pentingnya kata dalam korpus dokumen secara keseluruhan. Outputnya adalah kolom "features" yang berisi vektor fitur yang telah ditimbang dengan IDF.

In [ ]:
pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])
model = pipeline.fit(sentenceData)

* Pipeline(stages=[tokenizer, vectorizer, idf]): Membuat pipeline yang menggabungkan semua tahapan (tokenisasi, vektorisasi hitung, dan IDF) dalam satu alur kerja.
* pipeline.fit(sentenceData): Melatih pipeline dengan data sentenceData. Ini mencakup proses tokenisasi, pembuatan vektor hitung, dan perhitungan IDF, menghasilkan model yang sudah terlatih.

In [ ]:
import numpy as np

total_counts = model.transform(sentenceData)\
                    .select('rawFeatures').rdd\
                    .map(lambda row: row['rawFeatures'].toArray())\
                    .reduce(lambda x,y: [x[i]+y[i] for i in range(len(y))])

* Transformasi: Menggunakan model pipeline untuk menghasilkan vektor hitung dari data kalimat, kemudian menghitung total frekuensi setiap kata di seluruh dokumen.
  * model.transform(sentenceData): Menerapkan model pipeline yang telah dilatih pada sentenceData, menghasilkan DataFrame dengan kolom baru "rawFeatures" yang berisi vektor hitung fitur.
  * .select('rawFeatures'): Memilih kolom "rawFeatures" dari hasil transformasi.
  * .rdd: Mengonversi DataFrame menjadi RDD (Resilient Distributed Dataset).
  * .map(lambda row: row['rawFeatures'].toArray()): Mengonversi vektor fitur dari SparseVector menjadi array NumPy. Fungsi lambda ini diterapkan pada setiap baris.
  * .reduce(lambda x, y: [x[i] + y[i] for i in range(len(y))]): Menjumlahkan array vektor hitung untuk setiap kata, menghasilkan total frekuensi kata di seluruh dokumen.

In [ ]:
vocabList = model.stages[1].vocabulary

* model.stages[1].vocabulary: Mengambil daftar kosakata (vocabulary) dari CountVectorizer (stages[1] dalam pipeline), yang merupakan urutan kata-kata yang diindeks berdasarkan urutan kemunculannya.

In [ ]:
d = {'vocabList':vocabList,'counts':total_counts}

spark.createDataFrame(np.array(list(d.values())).T.tolist(),list(d.keys())).show()

* d = {'vocabList':vocabList, 'counts':total_counts}: Membuat dictionary d dengan kunci 'vocabList' dan 'counts', di mana 'vocabList' adalah daftar kosakata dan 'counts' adalah frekuensi total kata.
* np.array(list(d.values())).T.tolist(): Mengonversi dictionary ke dalam format array NumPy, mentransposkan array, dan kemudian mengonversi ke daftar Python (list) yang dapat digunakan untuk membuat DataFrame.
* spark.createDataFrame(..., list(d.keys())): Membuat DataFrame Spark dari data yang telah disiapkan, menggunakan kunci dictionary sebagai nama kolom.

In [ ]:
model.transform(sentenceData).show(truncate=True)

In [ ]:
model.transform(sentenceData).show(truncate=False)

* model.transform(sentenceData).show(truncate=True): Menampilkan DataFrame hasil transformasi dari sentenceData setelah pipeline diterapkan. truncate=True memastikan kolom yang panjang dipotong untuk tampilan yang lebih ringkas.

In [ ]:
from pyspark.sql.types import ArrayType, StringType

def termsIdx2Term(vocabulary):
    def termsIdx2Term(termIndices):
        return [vocabulary[int(index)] for index in termIndices]
    return udf(termsIdx2Term, ArrayType(StringType()))

Membuat fungsi mengonversi indeks kata menjadi kata asli:
* termsIdx2Term(vocabulary): Ini adalah fungsi yang menerima vocabulary (daftar kosakata) dan mengembalikan fungsi UDF (User Defined Function) PySpark.
* termsIdx2Term(termIndices): Ini adalah fungsi internal yang mengonversi indeks kata menjadi kata itu sendiri. termIndices adalah daftar indeks yang akan dikonversi.
* return [vocabulary[int(index)] for index in termIndices]: Mengonversi setiap indeks dalam termIndices menjadi kata menggunakan vocabulary dan mengembalikan daftar kata.
* return udf(termsIdx2Term, ArrayType(StringType())): Membuat UDF PySpark yang menggunakan fungsi termsIdx2Term. UDF ini akan digunakan untuk mengonversi vektor fitur menjadi daftar kata.

In [ ]:
vectorizerModel = model.stages[1]
vocabList = vectorizerModel.vocabulary
vocabList

* vectorizerModel = model.stages[1]: Mengambil model CountVectorizer dari pipeline yang telah dilatih.
* vocabList = vectorizerModel.vocabulary: Mengambil daftar kosakata dari CountVectorizer. Daftar ini berisi kata-kata yang digunakan dalam model vektorisasi hitung.

In [ ]:
rawFeatures = model.transform(sentenceData).select('rawFeatures')
rawFeatures.show()

* model.transform(sentenceData): Menerapkan model pipeline pada DataFrame sentenceData.
* .select('rawFeatures'): Memilih kolom rawFeatures yang berisi vektor hitung dari CountVectorizer.

## Predictive Analytics

Predictive Analytics adalah metode analitik yang menggunakan data, statistik, dan teknik machine learning untuk membuat prediksi tentang kejadian atau hasil masa depan. Ini melibatkan penggunaan data historis untuk membangun model yang dapat memproyeksikan tren, pola, atau nilai di masa depan. Tujuannya adalah untuk memberikan wawasan yang bermanfaat dan mendukung pengambilan keputusan berdasarkan prediksi yang dihasilkan.

Pada kali ini akan dicoba beberapa metode umum:
* Regresi: Memodelkan hubungan antara fitur dan variabel target untuk prediksi nilai kontinu atau probabilitas.
* Decision Tree: Menggunakan struktur pohon keputusan untuk memprediksi kelas atau nilai berdasarkan fitur.
* K-Means: Mengelompokkan data ke dalam klaster berdasarkan kesamaan fitur untuk segmentasi data.

### Regression

Regresi adalah teknik statistik yang digunakan untuk mengukur hubungan antara variabel independen (fitur) dan variabel dependen (target). Ini bertujuan untuk memprediksi nilai kontinu dari variabel dependen berdasarkan variabel independen.

Metode:
* Regresi Linier: Memprediksi nilai target sebagai kombinasi linier dari fitur-fitur. Contoh: Memprediksi harga rumah berdasarkan ukuran dan lokasi.
* Regresi Logistik: Digunakan untuk klasifikasi biner, memprediksi probabilitas dari dua kelas yang berbeda. Contoh: Memprediksi apakah email adalah spam atau bukan spam.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [ ]:
house_df = sqlContext.read.load('housing.csv',sep=" ", format="csv", inferSchema="true", header="true")
house_df.take(5)

* sqlContext.read.load(...): Membaca data dari file CSV housing.csv dengan pemisah spasi (sep=" "), menggunakan format CSV, dan secara otomatis menyimpulkan tipe data (inferSchema="true"). Baris pertama dianggap sebagai header (header="true").
* house_df.take(5): Mengambil dan menampilkan 5 baris pertama dari DataFrame untuk melihat data yang telah dimuat.

In [ ]:
house_df.cache()
house_df.printSchema()

* house_df.cache(): Menyimpan DataFrame di memori untuk meningkatkan performa jika DataFrame digunakan berkali-kali.
* house_df.printSchema(): Menampilkan skema DataFrame, yaitu struktur kolom dan tipe datanya.

In [ ]:
house_df.describe().toPandas().transpose()

* house_df.describe(): Menghitung statistik deskriptif seperti mean, min, max untuk setiap kolom numerik.
* .toPandas().transpose(): Mengonversi hasil statistik deskriptif ke DataFrame Pandas dan mentranspose-nya agar lebih mudah dibaca.

In [ ]:
type(house_df)

In [ ]:
import pandas as pd
numeric_features = [t[0] for t in house_df.dtypes if t[1] == 'int' or t[1] == 'double']
sampled_data = house_df.select(numeric_features).sample(False, 0.8).toPandas()
type(sampled_data)

* numeric_features: Menyaring kolom-kolom yang bertipe int atau double dari DataFrame.
* sample(False, 0.8): Mengambil sampel 80% dari data secara acak tanpa pengulangan.
* .toPandas(): Mengonversi hasil sampling ke DataFrame Pandas.
type(sampled_data): Mengecek tipe data dari sampled_data.

setelah menjadi datarame pandas, dapat dilakukan proses data analitik seperti pada python

In [ ]:
house_df.columns

* house_df.columns: Menampilkan nama-nama kolom dalam DataFrame.


In [ ]:
house_df.show(3)

* house_df.show(3): Menampilkan 3 baris pertama dari DataFrame untuk pemeriksaan.

In [ ]:
from pyspark.ml.feature import VectorAssembler
inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'RM', 'AGE',
             'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT', 'MV']
outputCol = "features"
df_va = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
df = df_va.transform(house_df)
df = df.select(['nox','features'])
newcolumns = ['label','features']
df.toDF(*newcolumns).show(truncate=False)

* VectorAssembler: Menggabungkan kolom fitur menjadi satu kolom fitur vektor (features).
* inputCols: Daftar kolom fitur yang akan digabungkan.
* outputCol: Nama kolom hasil gabungan fitur.
* df_va.transform(house_df): Menerapkan VectorAssembler untuk DataFrame house_df dan menghasilkan DataFrame baru dengan kolom features.
* df.select(['nox', 'features']): Memilih kolom nox sebagai label dan features sebagai kolom fitur.
* df.toDF(*newcolumns): Mengganti nama kolom menjadi label dan features.

In [ ]:
# Membagi data
(trainingData, testData) = df.randomSplit([0.7, 0.3])

* randomSplit([0.7, 0.3]): Membagi data menjadi dua subset: 70% untuk pelatihan (trainingData) dan 30% untuk pengujian (testData).

In [ ]:
trainingData.show()

In [ ]:
testData.show()

In [ ]:
from pyspark.ml.regression import LinearRegression
# Melatih model
lr = LinearRegression(featuresCol="features", labelCol="nox")
model = lr.fit(trainingData)

* LinearRegression: Membuat objek model regresi linier dengan kolom fitur (featuresCol) dan kolom label (labelCol).
* lr.fit(trainingData): Melatih model regresi linier dengan data pelatihan.

In [ ]:
# Melihat koefisien model
print("Coefficients: " + str(model.coefficients))

# Melihat intercept model
print("Intercept: " + str(model.intercept))

* model.coefficients: Menampilkan koefisien model.
* model.intercept: Menampilkan intercept model.

In [ ]:
# Mendapatkan ringkasan model
trainingSummary = model.summary
trainingSummary

* model.summary: Mengambil ringkasan model yang berisi metrik evaluasi seperti RMSE, R2, dll.

In [ ]:
# Memprediksi
predictions = model.transform(testData)

* model.transform(testData): Menerapkan model yang telah dilatih untuk membuat prediksi pada data pengujian (testData).

In [ ]:
predictions.show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
# Evaluasi
evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="nox")
evaluator.evaluate(predictions)

* model.transform(testData): Menerapkan model yang telah dilatih untuk membuat prediksi pada data pengujian (testData).
* predictions.show(): Menampilkan hasil prediksi.
* RegressionEvaluator: Mengukur kinerja model regresi.
* evaluator.evaluate(predictions): Mengevaluasi hasil prediksi menggunakan metrik seperti RMSE (nilai default).

### Decision Tree Classiication

Decision Tree adalah model yang membagi data menjadi subset berdasarkan fitur dengan membuat keputusan berbasis percabangan. Setiap node internal mewakili tes pada fitur, dan setiap cabang mewakili hasil tes. Daun akhir mewakili hasil prediksi.

Penerapan:
* Decision Tree untuk Klasifikasi: Memprediksi kelas dari data input berdasarkan fitur-fitur. Contoh: Memprediksi apakah seorang pelanggan akan membeli produk atau tidak.
* Decision Tree untuk Regresi: Memprediksi nilai kontinu dengan membagi data pada nilai fitur. Contoh: Memprediksi harga properti berdasarkan fitur-fitur seperti ukuran dan lokasi.

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer

data = df.toDF(*newcolumns)

In [ ]:
data.show()

* df.toDF(*newcolumns): Mengonversi DataFrame df dengan kolom yang dinamai ulang menjadi DataFrame Spark yang siap digunakan untuk model pembelajaran mesin.

In [ ]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

In [ ]:
labelIndexer

* StringIndexer: Mengonversi label kategorikal menjadi label numerik yang dapat diproses oleh model pembelajaran mesin.
* inputCol="label": Kolom asli yang berisi label kategorikal.
* outputCol="indexedLabel": Nama kolom hasil yang berisi label numerik.

In [ ]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)


* VectorIndexer: Mengidentifikasi fitur kategorikal dalam kolom fitur dan mengonversinya menjadi format numerik. Fitur dengan lebih dari 4 kategori dianggap sebagai fitur kontinu.
* inputCol="features": Kolom yang berisi fitur.
* outputCol="indexedFeatures": Nama kolom hasil yang berisi fitur yang diindeks.

In [ ]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

* randomSplit([0.7, 0.3]): Membagi data menjadi dua subset: 70% untuk pelatihan (trainingData) dan 30% untuk pengujian (testData).

In [ ]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

* DecisionTreeClassifier: Membuat model klasifikasi menggunakan pohon keputusan dengan kolom label yang diindeks dan fitur yang diindeks.
* Pipeline: Menggabungkan beberapa tahap pemrosesan dan pelatihan model ke dalam satu objek. Pipeline ini termasuk indeksasi label, indeksasi fitur, dan pelatihan model pohon keputusan.
* pipeline.fit(trainingData): Melatih model dengan data pelatihan, yang juga menerapkan langkah-langkah indeksasi.

In [ ]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

* model.transform(testData): Menggunakan model yang telah dilatih untuk memprediksi label pada data pengujian.
* predictions.select("prediction", "indexedLabel", "features").show(5): Menampilkan beberapa contoh hasil prediksi bersama dengan label yang sebenarnya dan fitur.

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Acccuracy = %g " % accuracy)
print("Test Error = %g " % (1.0 - accuracy))

* MulticlassClassificationEvaluator: Evaluasi model klasifikasi multikelas dengan metrik seperti akurasi.
* evaluator.evaluate(predictions): Menghitung akurasi model berdasarkan prediksi dan label yang sebenarnya.
* print("Accuracy = %g " % accuracy): Menampilkan akurasi model.
* print("Test Error = %g " % (1.0 - accuracy)): Menampilkan tingkat kesalahan model (1 - akurasi).

In [ ]:
treeModel = model.stages[2]
# summary only
print(treeModel)

* model.stages[2]: Mengambil model pohon keputusan dari tahap ketiga pipeline (karena labelIndexer dan featureIndexer adalah tahap pertama dan kedua).
* print(treeModel): Menampilkan ringkasan model pohon keputusan, termasuk struktur pohon dan informasi tentang aturan klasifikasi.

### K Means

K-Means adalah algoritma clustering yang digunakan untuk mengelompokkan data ke dalam sejumlah klaster yang telah ditentukan. Algoritma ini bekerja dengan membagi data ke dalam klaster yang meminimalkan jarak intra-klaster (jarak antara data dalam klaster) dan memaksimalkan jarak antar-klaster (jarak antara klaster yang berbeda).

In [ ]:
from pyspark.ml.clustering import KMeans

# Loads data.
dataset = spark.read.format("libsvm").load("sample_kmeans_data.txt")
dataset.show(truncate=False)

* spark.read.format("libsvm").load("sample_kmeans_data.txt"): Membaca data dari file sample_kmeans_data.txt dalam format LIBSVM, yang umumnya digunakan untuk data fitur yang sudah dinormalisasi dan dikodekan. Format LIBSVM adalah format file yang sering digunakan untuk data pembelajaran mesin.

In [ ]:
# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

* KMeans().setK(2).setSeed(1): Membuat objek KMeans dengan K=2, artinya algoritma akan mencoba membagi data menjadi 2 klaster. setSeed(1) digunakan untuk memastikan reprodusibilitas hasil.
* kmeans.fit(dataset): Melatih model KMeans menggunakan dataset yang telah dimuat.

In [ ]:
# Make predictions
predictions = model.transform(dataset)

* model.transform(dataset): Menggunakan model yang telah dilatih untuk memprediksi klaster untuk setiap baris data dalam dataset. Hasilnya adalah DataFrame yang mencakup kolom baru prediction yang menunjukkan klaster mana yang ditetapkan untuk setiap data.

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

* ClusteringEvaluator(): Membuat evaluator untuk klasterisasi yang dapat digunakan untuk menilai kualitas klaster.
* evaluator.evaluate(predictions): Menghitung skor Silhouette berdasarkan hasil prediksi klaster. Skor Silhouette adalah metrik untuk menilai seberapa baik data diklasifikasikan ke dalam klaster. Nilai mendekati 1 menunjukkan klaster yang baik, sedangkan nilai mendekati -1 menunjukkan klaster yang buruk.
* print("Silhouette with squared euclidean distance = " + str(silhouette)): Menampilkan skor Silhouette.

In [ ]:
# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

* model.clusterCenters(): Mengambil pusat klaster (centroid) yang ditemukan oleh model KMeans.
* print("Cluster Centers: "): Menampilkan pusat-pusat klaster.
* for center in centers: print(center): Menampilkan masing-masing pusat klaster.

## Terima kasih

In [ ]:
spark.stop()